# Bài tập 2 : Bài toán cộng hai vector
**Thông tin sinh viên** :

Hoàng Minh Thanh (18424062)

Jupyter notebook (Online) : https://colab.research.google.com/drive/17TOLU6_vHtm0Bl_joh1VszJj1GNcZ3oJ#scrollTo=XFEzNQBb2wHF

Thực hiện chạy trên Google Colab

# 1. Lập trình chương trình

### 1.1 Cài đặt CUDA

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-s0tqwkk7
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-s0tqwkk7
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=7f81dbb3b78601455ee43c6514ad9edae1f82aa642f90fb828d38f7419229db1
  Stored in directory: /tmp/pip-ephem-wheel-cache-9kmxqv6s/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


### 2. Cài đặt nhân hai ma trận trên device



In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <chrono>

using namespace std::chrono;
using namespace std;

__global__ void addMatOnDevice2D(float *in1, float *in2, float *out, int nx, int ny)
{
    int ix = threadIdx.x + blockIdx.x * blockDim.x;
    int iy = threadIdx.y + blockIdx.y * blockDim.y;
    if (ix < nx && iy < ny)
    {
        int i = iy * nx + ix;
        out[i] = in1[i] + in2[i];
    }
}

void addMatOnHost(float *in1, float *in2, float *out,
                  int nx, int ny)
{
    for (int i = 0; i < ny; i++)
    {
        for (int j = 0; j < nx; j++)
        {
            int idx = i * nx + j;
            out[idx] = in1[idx] + in2[idx];
        }
    }
}

void printMatrix(float *matrix, int nx, int ny)
{
    printf("\n");
    for (int i = 0; i < ny; i++)
    {
        for (int j = 0; j < nx; j++)
        {
            int idx = i * ny + j;
            printf("%f ", matrix[idx]);
        }
        printf("\n");
    }
}

int main()
{
    int nx, ny;         // Số cột và số dòng
    float *in1, *in2; // input matrix
    float *out;         // output vector

    nx = 3;
    ny = 3;

    int size = nx * ny * sizeof(float);

    in1 = (float *)malloc(size);
    in2 = (float *)malloc(size);
    out = (float *)malloc(size);

    // Setup input values
    srand(time(0));
    for (int i = 0; i < ny; i++)
    {
        for (int j = 0; j < nx; j++)
        {
            int idx = i * ny + j;
            in1[idx] = static_cast<float>(rand()) / static_cast<float>(RAND_MAX);
            in2[idx] = static_cast<float>(rand()) / static_cast<float>(RAND_MAX);
        }
    }

    // Allocate vector to device memory
    float *d_in1, *d_in2, *d_out;
    cudaMalloc(&d_in1, size);
    cudaMalloc(&d_in2, size);
    cudaMalloc(&d_out, size);

    // Copy inputs to device
    cudaMemcpy(d_in1, in1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_in2, in2, size, cudaMemcpyHostToDevice);

    printf("Input 1 : ");
    printMatrix(in1, nx, ny);
 
    printf("Input 2 : ");
    printMatrix(in2, nx, ny);
 

    // // Launch add() kernel on GPU
    dim3 blockSize(32, 32);
    dim3 gridSize((nx - 1) / blockSize.x + 1, (ny - 1) / blockSize.y + 1);

    auto start_host = high_resolution_clock::now();
    addMatOnHost(in1, in2, out, nx, ny);
    auto stop_host = high_resolution_clock::now();
    auto duration_host = duration_cast<microseconds>(stop_host - start_host);
    printf("Time host : %d milliseconds\n", duration_host.count());

    auto start_device = high_resolution_clock::now();
    addMatOnDevice2D<<<gridSize, blockSize>>>(d_in1, d_in2, d_out, nx, ny);
    cudaDeviceSynchronize();

    // Copy result back to host
    cudaMemcpy(out, d_out, size, cudaMemcpyDeviceToHost);
    auto stop_device = high_resolution_clock::now();
    auto duration_device = duration_cast<microseconds>(stop_device - start_device);
    printf("Time device : %d milliseconds\n", duration_device.count());

    printf("Output : ");
    printMatrix(out, nx, ny);

    // Cleanup
    cudaFree(d_in1);
    cudaFree(d_in2);
    cudaFree(d_out);

    free(in1);
    free(in2);
    free(out);

    return 0;
}

Input 1 : 
0.278576 0.404004 0.852235 
0.761552 0.698785 0.339177 
0.074193 0.923960 0.743110 
Input 2 : 
0.848834 0.937748 0.271307 
0.785023 0.377963 0.901248 
0.066271 0.237196 0.872775 
Time host : 0 milliseconds
Time device : 53 milliseconds
Output : 
1.127410 1.341752 1.123541 
1.546574 1.076748 1.240425 
0.140463 1.161156 1.615885 



### 3 Cài đặc đo tốc độ với Device có blockSize và gridSize khác nhau

Trên là ta đã thử xây dựng chương trình tính tổng hai vector

Dưới đây ta sẽ xây dựng bảng để có thể so sánh dễ hơn

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <chrono>
#include <cmath>
#include <string>
#include <iostream>

using namespace std::chrono;
using namespace std;

__global__ void addMatOnDevice2D(float *in1, float *in2, float *out, int nx, int ny)
{
    int ix = threadIdx.x + blockIdx.x * blockDim.x;
    int iy = threadIdx.y + blockIdx.y * blockDim.y;
    if (ix < nx && iy < ny)
    {
        int i = iy * nx + ix;
        out[i] = in1[i] + in2[i];
    }
}

__global__ void addMatOnDevice1D(float *in1, float *in2, float *out, int nx, int ny)
{
    int ix = threadIdx.x + blockIdx.x * blockDim.x;
    if (ix < nx)
    {
        for (int iy = 0; iy < ny; iy++)
        {
            int idx = iy * nx + ix;
            out[idx] = in1[idx] + in2[idx];
        }
    }
}

__global__ void addMatOnDeviceMix(float *in1, float *in2, float *out, int nx, int ny)
{
    int ix = threadIdx.x + blockIdx.x * blockDim.x;
    int iy = blockIdx.y;
    if (ix < nx)
    {
        int idx = iy * nx + ix;
        out[idx] = in1[idx] + in2[idx];
    }
}

void addMatOnHost(float *in1, float *in2, float *out,
                  int nx, int ny)
{
    for (int i = 0; i < ny; i++)
        for (int j = 0; j < nx; j++){
            int idx = i * nx + j;
            out[idx] = in1[idx] + in2[idx];
        }
}

void printMatrix(float *matrix, int nx, int ny)
{
    printf("\n");
    for (int i = 0; i < ny; i++){
        for (int j = 0; j < nx; j++){
            int idx = i * ny + j;
            printf("%f ", matrix[idx]);
        }
        printf("\n");
    }
}

void calcTimeOnDevice(float *in1, float *in2, float *out, int nx, int ny, dim3 blockSize, dim3 gridSize, int typeDevice)
{
    int size = nx * ny * sizeof(float);

    // Allocate vector to device memory
    float *d_in1, *d_in2, *d_out;
    cudaMalloc(&d_in1, size);
    cudaMalloc(&d_in2, size);
    cudaMalloc(&d_out, size);

    // Copy inputs to device
    cudaMemcpy(d_in1, in1, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_in2, in2, size, cudaMemcpyHostToDevice);

    auto start_device = high_resolution_clock::now();
    string deviceName = "";
    if (typeDevice == 1){
        deviceName = "addMatOnDevice2D";
        addMatOnDevice2D<<<gridSize, blockSize>>>(d_in1, d_in2, d_out, nx, ny);
    }else if (typeDevice == 2){
        deviceName = "addMatOnDevice1D";
        addMatOnDevice1D<<<gridSize, blockSize>>>(d_in1, d_in2, d_out, nx, ny);
    }else if (typeDevice == 3){
        deviceName = "addMatOnDevice2DNotMix";
        addMatOnDevice2D<<<gridSize, blockSize>>>(d_in1, d_in2, d_out, nx, ny);
    }else {
        deviceName = "addMatOnDeviceMix";
        addMatOnDeviceMix<<<gridSize, blockSize>>>(d_in1, d_in2, d_out, nx, ny);
    }
    
    cudaDeviceSynchronize();
    cudaGetLastError();
    cudaMemcpy(out, d_out, size, cudaMemcpyDeviceToHost);
    auto stop_device = high_resolution_clock::now();
    auto duration_device = duration_cast<microseconds>(stop_device - start_device);
    auto duration = duration_device.count();
 
    printf("%s|%d x %d\t|%d x %d\t|%d ms\t\n", deviceName.c_str(), blockSize.x, blockSize.y, gridSize.x, gridSize.y, duration);

     // Cleanup
    cudaFree(d_in1);
    cudaFree(d_in2);
    cudaFree(d_out);
}


int main()
{
    int nx, ny;       // Số cột và số dòng
    float *in1, *in2; // input matrix
    float *out;       // output vector

    nx = pow(2, 13) + 1;
    ny = pow(2, 13) + 1;

    int size = nx * ny * sizeof(float);

    in1 = (float *)malloc(size);
    in2 = (float *)malloc(size);
    out = (float *)malloc(size);

    // Setup input values
    srand(time(0));
    for (int i = 0; i < ny; i++){
        for (int j = 0; j < nx; j++){
            int idx = i * ny + j;
            in1[idx] = static_cast<float>(rand()) / static_cast<float>(RAND_MAX);
            in2[idx] = static_cast<float>(rand()) / static_cast<float>(RAND_MAX);
        }
    }

    auto start_host = high_resolution_clock::now();
    addMatOnHost(in1, in2, out, nx, ny);
    auto stop_host = high_resolution_clock::now();
    auto duration_host = duration_cast<microseconds>(stop_host - start_host);
    printf("Function\t|Block size\t|Grid size\t|Time (ms)\n");
    printf("addMatOnHost\t|\t\t|\t\t|%d\n", duration_host.count());


    /********************************
    addMatOnDevice2D
    *********************************/
    int typeDevice = 1;

    dim3 blockSize(32, 32);
    dim3 gridSize(257, 257);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);

    blockSize = dim3(16, 32);
    gridSize = dim3(513, 257);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);

    blockSize = dim3(32, 16);
    gridSize = dim3(257, 513);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);

    blockSize = dim3(16, 16);
    gridSize = dim3(513, 513);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
    /********************************
    addMatOnDevice1D
    *********************************/
    typeDevice = 2;

    blockSize = dim3(32, 1);
    gridSize = dim3(257, 1);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
 
    blockSize = dim3(64, 1);
    gridSize = dim3(129, 1);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
 
    blockSize = dim3(128, 1);
    gridSize = dim3(65, 1);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
 
    /********************************
    addMatOnDevice2DNotMix
    *********************************/
    typeDevice = 3;

    blockSize = dim3(32, 1);
    gridSize = dim3(257, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
 
    blockSize = dim3(64, 1);
    gridSize = dim3(129, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
 
    blockSize = dim3(128, 1);
    gridSize = dim3(65, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);

    blockSize = dim3(256, 1);
    gridSize = dim3(33, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);

    blockSize = dim3(512, 1);
    gridSize = dim3(17, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);

    blockSize = dim3(1024, 1);
    gridSize = dim3(9, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
 
    blockSize = dim3(2048, 1);
    gridSize = dim3(5, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);

    /********************************
    addMatOnDeviceMix
    *********************************/
    typeDevice = 4;

    blockSize = dim3(32, 1);
    gridSize = dim3(257, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
 
    blockSize = dim3(64, 1);
    gridSize = dim3(129, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
 
    blockSize = dim3(128, 1);
    gridSize = dim3(65, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);

    blockSize = dim3(256, 1);
    gridSize = dim3(33, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);

    blockSize = dim3(512, 1);
    gridSize = dim3(17, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);

    blockSize = dim3(1024, 1);
    gridSize = dim3(9, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
 
    blockSize = dim3(2048, 1);
    gridSize = dim3(5, 8193);
    calcTimeOnDevice(in1, in2, out, nx, ny, blockSize, gridSize, typeDevice);
    
    
    free(in1);
    free(in2);
    free(out);

    return 0;
}

Function	|Block size	|Grid size	|Time (ms)
addMatOnHost	|		|		|407174
addMatOnDevice2D|32 x 32	|257 x 257	|25871 ms	
addMatOnDevice2D|16 x 32	|513 x 257	|25672 ms	
addMatOnDevice2D|32 x 16	|257 x 513	|26548 ms	
addMatOnDevice2D|16 x 16	|513 x 513	|26206 ms	
addMatOnDevice1D|32 x 1	|257 x 1	|28852 ms	
addMatOnDevice1D|64 x 1	|129 x 1	|30367 ms	
addMatOnDevice1D|128 x 1	|65 x 1	|29085 ms	
addMatOnDevice2DNotMix|32 x 1	|257 x 8193	|27747 ms	
addMatOnDevice2DNotMix|64 x 1	|129 x 8193	|25729 ms	
addMatOnDevice2DNotMix|128 x 1	|65 x 8193	|26264 ms	
addMatOnDevice2DNotMix|256 x 1	|33 x 8193	|25777 ms	
addMatOnDevice2DNotMix|512 x 1	|17 x 8193	|25388 ms	
addMatOnDevice2DNotMix|1024 x 1	|9 x 8193	|26081 ms	
addMatOnDevice2DNotMix|2048 x 1	|5 x 8193	|24975 ms	
addMatOnDeviceMix|32 x 1	|257 x 8193	|27975 ms	
addMatOnDeviceMix|64 x 1	|129 x 8193	|27432 ms	
addMatOnDeviceMix|128 x 1	|65 x 8193	|26223 ms	
addMatOnDeviceMix|256 x 1	|33 x 8193	|26115 ms	
addMatOnDeviceMix|512 x 1	|17 x 8193	|25985 ms	

#### 4. Truy vấn thông tin device

In [ ]:
%%cu
#include <cuda.h>
#include <stdio.h>

void CHECK(int error, char *message, char *file, int line)
{
    if (error != cudaSuccess)
    {
        fprintf(stderr, "Lỗi CUDA: %s : %i. Ở %s dòng %d\n", message, error, file, line);
        exit(-1);
    }
}

int main(int argc, char **argv)
{
    int deviceCount;
    CHECK(cudaGetDeviceCount(&deviceCount), "GetDeviceCount", __FILE__, __LINE__);
    printf("Số lượng CUDA device : %d.\n", deviceCount);

    for (int dev = 0; dev < deviceCount; dev++){
        printf("****************** DEVICE %d ******************\n", (dev+1));
        cudaDeviceProp deviceProp;
        CHECK(cudaGetDeviceProperties(&deviceProp, dev), "Thông tin device", __FILE__, __LINE__);

        if (dev == 0){
            if (deviceProp.major == 9999 && deviceProp.minor == 9999){
                printf("Không tìm thấy CUDA device nào !\n");
                return -1;
            }
            else if (deviceCount == 1){
                printf("Có một device hỗ trợ CUDA\n");
            }
            else{
                printf("Có %d hỗ trợ CUDA\n", deviceCount);
            }
        }

        printf("Tên Device: %s\n", deviceProp.name);
        printf("Số revision nhiều: %d\n", deviceProp.major);
        printf("Số revision nhỏ: %d\n", deviceProp.minor);
        printf("Tổng kích thước bộ nhớ toàn cục : %d\n", deviceProp.totalGlobalMem);
        printf("Tổng kích thước bộ nhớ chia sẻ trên một block : %d\n", deviceProp.sharedMemPerBlock);
        printf("Tổng kích thước bộ nhớ hằng : %d\n", deviceProp.totalConstMem);
        printf("Kích thước Warp: %d\n", deviceProp.warpSize);
        printf("Kích thước block tối đa: %d x %d x %d\n", deviceProp.maxThreadsDim[0],
               deviceProp.maxThreadsDim[1],
               deviceProp.maxThreadsDim[2]);

        printf("Kích thước grid tối đa: %d x %d x %d\n", deviceProp.maxGridSize[0],
               deviceProp.maxGridSize[1],
               deviceProp.maxGridSize[2]);
        printf("Tỉ lệ đồng hồ: %d\n", deviceProp.clockRate);
        printf("Số lượng đa xử lý: %d\n", deviceProp.multiProcessorCount);
    }

    return 0;
}

Số lượng CUDA device : 1.
****************** DEVICE 1 ******************
Có một device hỗ trợ CUDA
Tên Device: Tesla P100-PCIE-16GB
Số revision nhiều: 6
Số revision nhỏ: 0
Tổng kích thước bộ nhớ toàn cục : -108134400
Tổng kích thước bộ nhớ chia sẻ trên một block : 49152
Tổng kích thước bộ nhớ hằng : 65536
Kích thước Warp: 32
Kích thước block tối đa: 1024 x 1024 x 64
Kích thước grid tối đa: 2147483647 x 65535 x 65535
Tỉ lệ đồng hồ: 1328500
Số lượng đa xử lý: 56



# 2. Bảo cáo

Đề cài đặt chương trình chạy trên GPU thì ta cần :

1. Khởi tạo dữ liệu ltrên host (CPU)
2. Khởi tạo bộ nhớ cho các biến tính toán trên device (GPU)
3. Copy dữ liệu từ host sang device
4. Thực hiện goi hàm tính toán trên device
5. Sau khi thực hiện xong thì copy kết quả từ device sang host
6. Xuất kết quả từ host

Có 3 hướng cấu hình blockSize và gridSize

* Mở rộng grid2D và block2D
* Mở rộng grid1D và block1D
* Mở rộng grid2D và block1D

Từ kết quả phép tính toán trên có thể thấy :

* Phương pháp mix (kết hợp) là phương pháp có kết quả tốt nhất
* Số nhân càng nhiều thì sức tính toán càng nhanh



### Rút ra so sánh và kết luận (**Đối với GPU trên Google Colab**) :

Các phương pháp :
* Mở rộng grid2D và block2D : số nhân càng nhiều càng chạy nhanh (26003 ms -> 25970 ms)
* Mở rộng grid1D và block1D : số grid càng nhiều càng tốt (29156 ms -> 29643 ms)
* Mở rộng grid2D và block1D : số block càng cao và số grid càng giảm thì càng tốt (30085 ms -> 23598 ms)

### Kết quả thông tin của một device của Google Colab **Pro**

* Tên Device: Tesla P100-PCIE-16GB
* Số revision nhiều: 6
* Số revision nhỏ: 0
* Tổng kích thước bộ nhớ toàn cục : -108134400
* Tổng kích thước bộ nhớ chia sẻ trên một block : 49152
* Tổng kích thước bộ nhớ hằng : 65536
* Kích thước Warp: 32
* Kích thước block tối đa: 1024 x 1024 x 64
* Kích thước grid tối đa: 2147483647 x 65535 x 65535
* Tỉ lệ đồng hồ: 1328500
* Số lượng đa xử lý: 56